In [74]:
from Model import *
from DataLoader import *
from Setting import *
import collections
import os
import numpy as np
import pickle
import heapq
import math

## Week3 code1 Median Maintenance & Heap Implementation
This week I implement a heap class with two methods "Insert" and "popmin" with O(log(n)) time complicity. And then, using heap class I implemented to construct an Median Maintenance algorithm, which could maintenance the median when a stream of numbers pour in. That is, when a new number come, it will compute a new median with in O(log(n)) time.

Coursera link:https://www.coursera.org/learn/algorithms-graphs-data-structures/lecture/iIzo8/heaps-operations-and-applications

Sildes of this algorithm:https://d18ky98rnyall9.cloudfront.net/_d0c402bcb519748dade72800c64d654a_slides_algo-ds-heaps-basics_typed.pdf?Expires=1572652800&Signature=Pqe1gROd2OWxITiXnhcGPns02vcXLC-a~GUmj6QOYbJaO4eiXiak7gBXIIKuUzxo8AdKmznz2ph5xv-WT520aUjuGMBK8JEDzlD9RQK~D65Br-EyppTyRuiZXfIzFBWZRgxYzkmrkDZMX~Rm5qPGf2sRx9fMw9qXlro-gk17AC8_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A


In [2]:
# heap class, using heapq package
class Heap(object):
    def __init__(self, initial=None, key=lambda x:x , reverse = False):
        self.key = key
        if initial:
            self._data = [(key(item),id(item), item) for item in initial]
            heapq.heapify(self._data)
        else:
            self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item),id(item), item))

    def pop(self):
        try:
            return heapq.heappop(self._data)[2]
        except:
            return None

    def length(self):
        return len(self._data)

In [301]:
# Heap class implementation using list
# two methods: log(n) insert,and log(n) popmin.
class Heap(object): 
    
    def __init__(self,maxflag = False):
        self._data = []
        self.maxflag = maxflag
        

    def insert(self, item):
        if self.maxflag == True:
            item = - item
        
        self._data.append(item)
        location = len(self._data)
        
        
        
        while item < self._data[location//2 -1] and location//2 -1>=0:
            self._data[location-1], self._data[location//2 -1] = self._data[location//2 -1],self._data[location-1] 
            location = location//2
    
    
    def findmin(self,loc):

        if (loc+1) * 2 >= len(self._data):
            return (loc+1)*2 -1
        else:
            if self._data[(loc+1)*2 -1] <= self._data[(loc+1)*2]:
                return (loc+1)*2 -1
            else:
                return (loc+1)*2
    
    

    def popmin(self):
        minvalue = self._data[0]
        
        self._data[0], self._data[len(self._data)-1] = self._data[len(self._data)-1], self._data[0]
        self._data = self._data[:-1]
        
        loc = 0
        while 2*(loc+1) <= len(self._data):
            minchild = self.findmin(loc)            
            if  self._data[loc]> self._data[minchild]:
                self._data[loc], self._data[minchild] = self._data[minchild], self._data[loc]
                loc = minchild
            else:
                break
        if self.maxflag == True:
            return -minvalue
        
        else:
            return  minvalue
        
    def getlength(self):
        return len(self._data)


# #test case1
# heap = Heap()
# for i in [4,4,8,9,4,12,9,11,13]:
#     heap.insert(i)
# heap.insert(1)
# heap.insert(2)
# heap.insert(200)
# heap.insert(11)
# assert heap.popmin() == 1
# assert heap.popmin() == 2
# assert heap.popmin() == 4
# assert heap.popmin() == 4
# assert heap.popmin() == 4
# assert heap.popmin() == 8


#test case2
# heap = Heap()
# alist = [5,5,5,4,12,6,8,2,3,1,2,3,54,1,23,12,23,21,23,5,6,7,8,92]
# for i in alist:
#     heap.insert(i)
# alist = sorted(alist)
# for i in range(len(alist)):
#     assert (heap.popmin() == alist[i])


#test case3
# import random
# alist = random.sample(range(1,1000), 200)
# for i in alist:
#     heap.insert(i)
# alist = sorted(alist)
# for i in range(len(alist)):
#     assert (heap.popmin() == alist[i])


In [302]:
class TwoHeapsMedian(Model):
    def __init__(self):
        super().__init__()

        
    def preprocess(self):
        self.data = self.dataLoader.data
        self.data = [i[0] for i in self.data if i!=[]]
        self.codedict = {}
    
        
    def model(self):
        
        maxheap = Heap(maxflag = True)
        minheap = Heap(maxflag = False)

        medians = []
        alist = self.data

        min_value = -1e7
        max_value = 1e7
        count = 0


        for i in alist:
            count+=1
            print('count:',count,end ='')
            print(",in:",i,end= ', ')
            minl = minheap.getlength()
            maxl = maxheap.getlength()


            if i> min_value and (min_value!=-1e7 or count ==1):
                minheap.insert(i)
                if minl >= maxl:
                    maxheap.insert(minheap.popmin())

            elif i<max_value and (max_value!=1e7 or count ==1):
                maxheap.insert(i)
                if minl <= maxl:
                    minheap.insert(maxheap.popmin())

            elif i<= min_value and i>=max_value:
                if minl >= maxl:
                    maxheap.insert(i)

                elif minl <= maxl:
                    minheap.insert(i)

            if maxheap.getlength()>0:
                max_value = -maxheap._data[0]
            if minheap.getlength()>0:
                min_value = minheap._data[0]

            minl = minheap.getlength()
            maxl = maxheap.getlength()

            if count%2 == 0:
                print("median:",-maxheap._data[0],end= '')
                median = -maxheap._data[0]
            else:
                if maxl > minl:
                    median = -maxheap._data[0]
                    print("median:",-maxheap._data[0],end= '')
                else:
                    median = minheap._data[0]
                    print("median:",minheap._data[0],end= '')

            medians.append(median)
            print(",sum:",sum(medians))
        return(sum(medians)%10000)


In [303]:
processLine = lambda x : list(map(int,map(str.strip,x.split())))
arg = {'fileName':'./data/median.txt','numLines':None ,  'processLine' : processLine  }

d = DataLoader(**arg)
m = TwoHeapsMedian()
s = Setting(d,m,False)
s.run()

Loading data ...
Total 10000 lines read
--------------------------------------------------
count: 1,in: 6331, median: 6331,sum: 6331
count: 2,in: 2793, median: 2793,sum: 9124
count: 3,in: 1640, median: 2793,sum: 11917
count: 4,in: 9290, median: 2793,sum: 14710
count: 5,in: 225, median: 2793,sum: 17503
count: 6,in: 625, median: 1640,sum: 19143
count: 7,in: 6195, median: 2793,sum: 21936
count: 8,in: 2303, median: 2303,sum: 24239
count: 9,in: 5685, median: 2793,sum: 27032
count: 10,in: 1354, median: 2303,sum: 29335
count: 11,in: 4292, median: 2793,sum: 32128
count: 12,in: 7600, median: 2793,sum: 34921
count: 13,in: 6447, median: 4292,sum: 39213
count: 14,in: 4479, median: 4292,sum: 43505
count: 15,in: 9046, median: 4479,sum: 47984
count: 16,in: 7293, median: 4479,sum: 52463
count: 17,in: 5147, median: 5147,sum: 57610
count: 18,in: 1260, median: 4479,sum: 62089
count: 19,in: 1386, median: 4479,sum: 66568
count: 20,in: 6193, median: 4479,sum: 71047
count: 21,in: 4135, median: 4479,sum: 7552

count: 294,in: 936, median: 4229,sum: 1180645
count: 295,in: 2980, median: 4229,sum: 1184874
count: 296,in: 301, median: 4200,sum: 1189074
count: 297,in: 2625, median: 4200,sum: 1193274
count: 298,in: 8674, median: 4200,sum: 1197474
count: 299,in: 6863, median: 4229,sum: 1201703
count: 300,in: 2108, median: 4200,sum: 1205903
count: 301,in: 4466, median: 4229,sum: 1210132
count: 302,in: 6346, median: 4229,sum: 1214361
count: 303,in: 5193, median: 4289,sum: 1218650
count: 304,in: 8775, median: 4289,sum: 1222939
count: 305,in: 5247, median: 4292,sum: 1227231
count: 306,in: 631, median: 4289,sum: 1231520
count: 307,in: 9256, median: 4292,sum: 1235812
count: 308,in: 4561, median: 4292,sum: 1240104
count: 309,in: 3489, median: 4292,sum: 1244396
count: 310,in: 5787, median: 4292,sum: 1248688
count: 311,in: 1797, median: 4292,sum: 1252980
count: 312,in: 251, median: 4289,sum: 1257269
count: 313,in: 3095, median: 4289,sum: 1261558
count: 314,in: 2239, median: 4229,sum: 1265787
count: 315,in: 19

count: 607,in: 1847, median: 4277,sum: 2511905
count: 608,in: 9924, median: 4277,sum: 2516182
count: 609,in: 7681, median: 4289,sum: 2520471
count: 610,in: 2976, median: 4277,sum: 2524748
count: 611,in: 4170, median: 4277,sum: 2529025
count: 612,in: 9486, median: 4277,sum: 2533302
count: 613,in: 7184, median: 4289,sum: 2537591
count: 614,in: 7698, median: 4289,sum: 2541880
count: 615,in: 1307, median: 4289,sum: 2546169
count: 616,in: 4501, median: 4289,sum: 2550458
count: 617,in: 1213, median: 4289,sum: 2554747
count: 618,in: 495, median: 4277,sum: 2559024
count: 619,in: 9227, median: 4289,sum: 2563313
count: 620,in: 2120, median: 4277,sum: 2567590
count: 621,in: 4462, median: 4289,sum: 2571879
count: 622,in: 4658, median: 4289,sum: 2576168
count: 623,in: 5979, median: 4292,sum: 2580460
count: 624,in: 1203, median: 4289,sum: 2584749
count: 625,in: 8236, median: 4292,sum: 2589041
count: 626,in: 4308, median: 4292,sum: 2593333
count: 627,in: 1784, median: 4292,sum: 2597625
count: 628,in:

count: 919,in: 8194, median: 4292,sum: 3853556
count: 920,in: 5154, median: 4292,sum: 3857848
count: 921,in: 4780, median: 4308,sum: 3862156
count: 922,in: 803, median: 4292,sum: 3866448
count: 923,in: 9314, median: 4308,sum: 3870756
count: 924,in: 3132, median: 4292,sum: 3875048
count: 925,in: 6386, median: 4308,sum: 3879356
count: 926,in: 3938, median: 4292,sum: 3883648
count: 927,in: 1425, median: 4292,sum: 3887940
count: 928,in: 2186, median: 4289,sum: 3892229
count: 929,in: 8411, median: 4292,sum: 3896521
count: 930,in: 2802, median: 4289,sum: 3900810
count: 931,in: 8175, median: 4292,sum: 3905102
count: 932,in: 1025, median: 4289,sum: 3909391
count: 933,in: 6882, median: 4292,sum: 3913683
count: 934,in: 1391, median: 4289,sum: 3917972
count: 935,in: 2879, median: 4289,sum: 3922261
count: 936,in: 8542, median: 4289,sum: 3926550
count: 937,in: 526, median: 4289,sum: 3930839
count: 938,in: 5398, median: 4289,sum: 3935128
count: 939,in: 6189, median: 4292,sum: 3939420
count: 940,in: 

count: 1232,in: 9336, median: 4352,sum: 5204297
count: 1233,in: 8373, median: 4357,sum: 5208654
count: 1234,in: 7902, median: 4357,sum: 5213011
count: 1235,in: 2020, median: 4357,sum: 5217368
count: 1236,in: 8378, median: 4357,sum: 5221725
count: 1237,in: 7290, median: 4358,sum: 5226083
count: 1238,in: 3823, median: 4357,sum: 5230440
count: 1239,in: 5110, median: 4358,sum: 5234798
count: 1240,in: 3313, median: 4357,sum: 5239155
count: 1241,in: 5405, median: 4358,sum: 5243513
count: 1242,in: 8446, median: 4358,sum: 5247871
count: 1243,in: 4257, median: 4358,sum: 5252229
count: 1244,in: 1275, median: 4357,sum: 5256586
count: 1245,in: 2348, median: 4357,sum: 5260943
count: 1246,in: 3941, median: 4352,sum: 5265295
count: 1247,in: 2365, median: 4352,sum: 5269647
count: 1248,in: 9340, median: 4352,sum: 5273999
count: 1249,in: 1, median: 4352,sum: 5278351
count: 1250,in: 2271, median: 4351,sum: 5282702
count: 1251,in: 7818, median: 4352,sum: 5287054
count: 1252,in: 2359, median: 4351,sum: 529

...

count: 8543,in: 9958, median: 4966,sum: 39556211
count: 8544,in: 4012, median: 4965,sum: 39561176
count: 8545,in: 88, median: 4965,sum: 39566141
count: 8546,in: 6868, median: 4965,sum: 39571106
count: 8547,in: 789, median: 4965,sum: 39576071
count: 8548,in: 4383, median: 4964,sum: 39581035
count: 8549,in: 2274, median: 4964,sum: 39585999
count: 8550,in: 4203, median: 4962,sum: 39590961
count: 8551,in: 9481, median: 4964,sum: 39595925
count: 8552,in: 5743, median: 4964,sum: 39600889
count: 8553,in: 3721, median: 4964,sum: 39605853
count: 8554,in: 5007, median: 4964,sum: 39610817
count: 8555,in: 6901, median: 4965,sum: 39615782
count: 8556,in: 9865, median: 4965,sum: 39620747
count: 8557,in: 7840, median: 4966,sum: 39625713
count: 8558,in: 4897, median: 4965,sum: 39630678
count: 8559,in: 2114, median: 4965,sum: 39635643
count: 8560,in: 5089, median: 4965,sum: 39640608
count: 8561,in: 2156, median: 4965,sum: 39645573
count: 8562,in: 1724, median: 4964,sum: 39650537
count: 8563,in: 8094, m

count: 8855,in: 9720, median: 4972,sum: 41106321
count: 8856,in: 8975, median: 4972,sum: 41111293
count: 8857,in: 2019, median: 4972,sum: 41116265
count: 8858,in: 7694, median: 4972,sum: 41121237
count: 8859,in: 9145, median: 4973,sum: 41126210
count: 8860,in: 9010, median: 4973,sum: 41131183
count: 8861,in: 7906, median: 4974,sum: 41136157
count: 8862,in: 1244, median: 4973,sum: 41141130
count: 8863,in: 7460, median: 4974,sum: 41146104
count: 8864,in: 9995, median: 4974,sum: 41151078
count: 8865,in: 4684, median: 4974,sum: 41156052
count: 8866,in: 4331, median: 4973,sum: 41161025
count: 8867,in: 9332, median: 4974,sum: 41165999
count: 8868,in: 3943, median: 4973,sum: 41170972
count: 8869,in: 8896, median: 4974,sum: 41175946
count: 8870,in: 916, median: 4973,sum: 41180919
count: 8871,in: 2043, median: 4973,sum: 41185892
count: 8872,in: 6165, median: 4973,sum: 41190865
count: 8873,in: 2725, median: 4973,sum: 41195838
count: 8874,in: 9228, median: 4973,sum: 41200811
count: 8875,in: 1485,

count: 9168,in: 861, median: 4998,sum: 42668150
count: 9169,in: 9189, median: 4999,sum: 42673149
count: 9170,in: 3394, median: 4998,sum: 42678147
count: 9171,in: 1210, median: 4998,sum: 42683145
count: 9172,in: 5273, median: 4998,sum: 42688143
count: 9173,in: 4718, median: 4998,sum: 42693141
count: 9174,in: 1283, median: 4996,sum: 42698137
count: 9175,in: 9042, median: 4998,sum: 42703135
count: 9176,in: 7639, median: 4998,sum: 42708133
count: 9177,in: 6642, median: 4999,sum: 42713132
count: 9178,in: 8834, median: 4999,sum: 42718131
count: 9179,in: 9641, median: 5001,sum: 42723132
count: 9180,in: 5191, median: 5001,sum: 42728133
count: 9181,in: 4610, median: 5001,sum: 42733134
count: 9182,in: 6804, median: 5001,sum: 42738135
count: 9183,in: 5613, median: 5002,sum: 42743137
count: 9184,in: 6296, median: 5002,sum: 42748139
count: 9185,in: 9012, median: 5003,sum: 42753142
count: 9186,in: 9796, median: 5003,sum: 42758145
count: 9187,in: 9913, median: 5005,sum: 42763150
count: 9188,in: 4460,

count: 9480,in: 3508, median: 5002,sum: 44228919
count: 9481,in: 7182, median: 5003,sum: 44233922
count: 9482,in: 1815, median: 5002,sum: 44238924
count: 9483,in: 6451, median: 5003,sum: 44243927
count: 9484,in: 7384, median: 5003,sum: 44248930
count: 9485,in: 6309, median: 5005,sum: 44253935
count: 9486,in: 2502, median: 5003,sum: 44258938
count: 9487,in: 10000, median: 5005,sum: 44263943
count: 9488,in: 7689, median: 5005,sum: 44268948
count: 9489,in: 4051, median: 5005,sum: 44273953
count: 9490,in: 8354, median: 5005,sum: 44278958
count: 9491,in: 2104, median: 5005,sum: 44283963
count: 9492,in: 4107, median: 5003,sum: 44288966
count: 9493,in: 9940, median: 5005,sum: 44293971
count: 9494,in: 9716, median: 5005,sum: 44298976
count: 9495,in: 5004, median: 5005,sum: 44303981
count: 9496,in: 7083, median: 5005,sum: 44308986
count: 9497,in: 8085, median: 5006,sum: 44313992
count: 9498,in: 145, median: 5005,sum: 44318997
count: 9499,in: 9569, median: 5006,sum: 44324003
count: 9500,in: 1317

count: 9792,in: 2237, median: 5002,sum: 45790898
count: 9793,in: 49, median: 5002,sum: 45795900
count: 9794,in: 214, median: 5001,sum: 45800901
count: 9795,in: 7772, median: 5002,sum: 45805903
count: 9796,in: 112, median: 5001,sum: 45810904
count: 9797,in: 5851, median: 5002,sum: 45815906
count: 9798,in: 7861, median: 5002,sum: 45820908
count: 9799,in: 4997, median: 5002,sum: 45825910
count: 9800,in: 4980, median: 5001,sum: 45830911
count: 9801,in: 9135, median: 5002,sum: 45835913
count: 9802,in: 3978, median: 5001,sum: 45840914
count: 9803,in: 737, median: 5001,sum: 45845915
count: 9804,in: 2984, median: 5000,sum: 45850915
count: 9805,in: 9689, median: 5001,sum: 45855916
count: 9806,in: 507, median: 5000,sum: 45860916
count: 9807,in: 3601, median: 5000,sum: 45865916
count: 9808,in: 3440, median: 4999,sum: 45870915
count: 9809,in: 3498, median: 4999,sum: 45875914
count: 9810,in: 6164, median: 4999,sum: 45880913
count: 9811,in: 975, median: 4999,sum: 45885912
count: 9812,in: 5198, media